In [3]:
import datetime
import os
import sys

import pandas as pd

from py_kd_data_common_proj.ds_utils.ds_common_util import init_log_config, init_pandas_setting
from py_kd_data_common_proj.ds_utils.ds_date_util import format_date, delta_time, make_stat_week_day_list

### 出租车上下车信息
    measure: 
        total_amount, tip_amount, tolls_amount, fare_amount, passenger_count
    dim:
        tpep_pickup_datetime: 2020-12-01 00:07:13
        tpep_dropoff_datetime: 2020-12-01 00:18:12
        PULocationID: 138
        DOLocationID: 263

In [4]:
taxi_activity_file_12 = "/Users/qian.jiang/Documents/_AllDocMap/05_Dataset/kylin_demo/data/trip_data_2018-2021/2020/2020-12.csv"
taxi_activity_file_11 = "/Users/qian.jiang/Documents/_AllDocMap/05_Dataset/kylin_demo/data/trip_data_2018-2021/2020/2020-11.csv"
taxi_activity_file_10 = "/Users/qian.jiang/Documents/_AllDocMap/05_Dataset/kylin_demo/data/trip_data_2018-2021/2020/2020-10.csv"
taxi_activity_file_09 = "/Users/qian.jiang/Documents/_AllDocMap/05_Dataset/kylin_demo/data/trip_data_2018-2021/2020/2020-09.csv"
taxi_activity_file_08 = "/Users/qian.jiang/Documents/_AllDocMap/05_Dataset/kylin_demo/data/trip_data_2018-2021/2020/2020-08.csv"
taxi_activity_file_07 = "/Users/qian.jiang/Documents/_AllDocMap/05_Dataset/kylin_demo/data/trip_data_2018-2021/2020/2020-07.csv"
taxi_activity_file_01 = "/Users/qian.jiang/Documents/_AllDocMap/05_Dataset/kylin_demo/data/trip_data_2018-2021/2021/2021-01.csv"
taxi_activity_file_02 = "/Users/qian.jiang/Documents/_AllDocMap/05_Dataset/kylin_demo/data/trip_data_2018-2021/2021/2021-02.csv"
taxi_activity_file_03 = "/Users/qian.jiang/Documents/_AllDocMap/05_Dataset/kylin_demo/data/trip_data_2018-2021/2021/2021-03.csv"
taxi_activity_file_04 = "/Users/qian.jiang/Documents/_AllDocMap/05_Dataset/kylin_demo/data/trip_data_2018-2021/2021/2021-04.csv"
taxi_activity_file_05 = "/Users/qian.jiang/Documents/_AllDocMap/05_Dataset/kylin_demo/data/trip_data_2018-2021/2021/2021-05.csv"
taxi_activity_file_06 = "/Users/qian.jiang/Documents/_AllDocMap/05_Dataset/kylin_demo/data/trip_data_2018-2021/2021/2021-06.csv"

# taxi_df_12 = pd.read_csv(taxi_activity_file_12, low_memory=False)
# taxi_df_11 = pd.read_csv(taxi_activity_file_11, low_memory=False)
# taxi_df_10 = pd.read_csv(taxi_activity_file_10, low_memory=False)
# taxi_df_09 = pd.read_csv(taxi_activity_file_09, low_memory=False)
# taxi_df_08 = pd.read_csv(taxi_activity_file_08, low_memory=False)
# taxi_df_07 = pd.read_csv(taxi_activity_file_07, low_memory=False)
taxi_df_06 = pd.read_csv(taxi_activity_file_06, low_memory=False)
taxi_df_05 = pd.read_csv(taxi_activity_file_05, low_memory=False)
taxi_df_04 = pd.read_csv(taxi_activity_file_04, low_memory=False)
taxi_df_03 = pd.read_csv(taxi_activity_file_03, low_memory=False)
taxi_df_02 = pd.read_csv(taxi_activity_file_02, low_memory=False)
taxi_df_01 = pd.read_csv(taxi_activity_file_01, low_memory=False)

# taxi_df = pd.concat(
#     [taxi_df_01, taxi_df_02, taxi_df_03, taxi_df_04, taxi_df_05, taxi_df_06, taxi_df_07, taxi_df_08, taxi_df_09,
#      taxi_df_10, taxi_df_11,
#      taxi_df_12], axis=0)

taxi_df = pd.concat(
    [taxi_df_01, taxi_df_02, taxi_df_03, taxi_df_04, taxi_df_05, taxi_df_06], axis=0)

In [ ]:
taxi_df['pickup_date'] = taxi_df['tpep_pickup_datetime'].map(
    lambda x: str(format_date(x, new_pattern="%Y-%m-%d")))

taxi_df['dropoff_date'] = taxi_df['tpep_dropoff_datetime'].map(
    lambda x: str(format_date(x, new_pattern="%Y-%m-%d")))

taxi_df['elapse_time'] = taxi_df.apply(lambda x: str(delta_time(x['tpep_pickup_datetime'], x['tpep_dropoff_datetime'])),
                                       axis=1)

taxi_df

### 出租车活动区域
    dim：
        LocationID: 138

In [ ]:
taxi_zone_file = "/Users/qian.jiang/workspace4py/JamesPython/_data/kylin_demo/data/taxi_zone/taxi_zone_lookup.csv"

taxi_zone_df = pd.read_csv(taxi_zone_file, low_memory=False)

taxi_zone_df

### 连接出租车上下车活动信息与活动区域


In [ ]:
taxi_pickup_df = taxi_df.join(taxi_zone_df.set_index('LocationID'), how='left', on='PULocationID',
                              lsuffix='_L',
                              rsuffix="_R")

taxi_pickup_df = taxi_pickup_df.join(taxi_zone_df.set_index('LocationID'), how='left', on='DOLocationID',
                                     lsuffix='_PU',
                                     rsuffix="_DO")

taxi_pickup_df

### 选取第 9999 行

In [ ]:
taxi_pickup_df.iloc[9999]

### 选取第 9999 行的指定列

In [ ]:
taxi_pickup_df.loc[
    9999, ['pickup_date', 'total_amount', 'trip_distance', 'passenger_count', 'elapse_time', 'tpep_pickup_datetime',
           'pickup_date', 'Zone_PU', 'tpep_dropoff_datetime', 'dropoff_date', 'Zone_DO']]

In [ ]:
taxi_pickup_df.loc[:,
['pickup_date', 'total_amount', 'trip_distance', 'passenger_count', 'elapse_time', 'tpep_pickup_datetime',
 'pickup_date', 'Zone_PU', 'tpep_dropoff_datetime', 'dropoff_date', 'Zone_DO']]

### 日期维表
    day_start date,
    month_start date,
    quarter_start date,
    year_start date,
    month_en string,
    day_of_year int,
    week_of_year int

In [ ]:
calendar_file = "/Users/qian.jiang/workspace4py/JamesPython/_data/kylin_demo/data/lookup_calendar/kylin_cal.csv"

cal_df = pd.read_csv(calendar_file, low_memory=False)

cal_df

### 连接出租车上下车信息：上下车信息、区域、日期

In [ ]:
taxi_pickup_df = taxi_pickup_df.join(cal_df.set_index('day_start'), how='left', on='pickup_date',
                                     lsuffix='_L',
                                     rsuffix="_R")

taxi_pickup_df

In [ ]:
### 按照日期排序

In [ ]:
taxi_pickup_df = taxi_pickup_df.sort_values(axis=0, by=['pickup_date'], ascending=True)

taxi_pickup_df

In [ ]:
daily_count_df = taxi_pickup_df.groupby(['pickup_date', 'Zone_PU'])['total_amount'].count()
daily_count_df = daily_count_df.reset_index(name='cnt')
daily_count_df

In [ ]:
sort_by_cnt_df = daily_count_df.groupby(['Zone_PU'])['cnt'].count()
sort_by_cnt_df = sort_by_cnt_df.reset_index(name='cnt')
sort_by_cnt_df = sort_by_cnt_df.sort_values(axis=0, by=['cnt'], ascending=False)
sort_by_cnt_df

In [ ]:
all_dims = list(daily_count_df['Zone_PU'].unique())

# all_dims

### 绘制趋势图

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import zscore

# 赋值 dataframe
data = daily_count_df

data_week = data.copy()

TITLE = 'dim'
Y_LABEL = 'Taxi Activities'
fig = plt.figure(figsize=(10, 6))

colors = ['deepskyblue', 'blueviolet', 'peru', 'brown', 'black', 'red', 'gold', 'darkorange']

DIM_DATE_NAME = 'pickup_date'
DIM_NAME = 'Zone_PU'
MES_CNT_NAME = 'cnt'

dims = all_dims
dims = ['East New York']

# 生成周同比数据
stat_week_day_str_list = make_stat_week_day_list('2021-07-30', -1, 7, 52)

if len(stat_week_day_str_list) != 0:
    data_week = data[data.pickup_date.isin(stat_week_day_str_list)]

data_week = data_week.groupby([DIM_DATE_NAME, DIM_NAME])[MES_CNT_NAME].sum()
data_week = data_week.reset_index(name='cnt')

# 绘图
i = 0
for dim in dims:
    subset = data[data.Zone_PU.isin([dim])]
    subset = subset.sort_values(axis=0, by=[DIM_DATE_NAME, DIM_NAME], ascending=True)
    print('-' * 60)
    cnt_zscore = zscore(subset[MES_CNT_NAME])
    subset['cnt_zscore'] = cnt_zscore
    print(subset)
    print('')
    plt.plot(subset[DIM_DATE_NAME], subset[MES_CNT_NAME], c=colors[i], label=f"{dim}-daily")
    i = (i + 1) % 8

    subset_week = data_week[data_week.Zone_PU.isin([dim])]
    subset_week = subset_week.sort_values(axis=0, by=[DIM_DATE_NAME, DIM_NAME], ascending=True)
    print('-' * 60)
    cnt_zscore_week = zscore(subset_week[MES_CNT_NAME])
    subset_week['cnt_zscore'] = cnt_zscore_week
    print(subset_week)
    print('')
    plt.plot(subset_week[DIM_DATE_NAME], subset_week[MES_CNT_NAME], c=colors[i], label=f"{dim}-weekly")
    i = (i + 1) % 8

plt.legend(loc='upper left')
plt.title(TITLE)
plt.ylabel(Y_LABEL)
plt.xlabel('date')
plt.xticks(rotation=60)
plt.show()